# データベースの作成

In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [2]:
# データの読み込み
# data = pd.read_csv("../../gci_project_storage/mynavi_data2.csv", sep=",", encoding="UTF-8")
# 上手くいかないので、pickle
import pickle
with open('../../gci_project_storage/data.pickle', mode='rb') as f:
     data = pickle.load(f)

In [3]:
data.shape

(46469, 224)

<h3>目次：</h3><br>
・Geo Coding用のカラムの追加<br>
・データベースにぶち込む<br>

<h3>Geo Coding用のカラムの追加

In [4]:
new_colnames = ["altitude","latitude"]
df_new = pd.DataFrame(index=data.index, columns=new_colnames)
data = pd.concat([data,df_new],axis=1)
data.shape

(46469, 226)

<h3>データベースにぶち込む

In [5]:
# データベースの情報
server = 'gciteam16.database.windows.net'
database = 'mynavi-database'
username = 'gciteam16'
password = 'Password0'
port =1433
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [6]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Aug 29 2017 13:06:11 
	Copyright (C) 2017 Microsoft Corporation



In [9]:
# 接続エンジンの作成
engine_config = "?driver=ODBC+Driver+13+for+SQL+Server?charset=shift-jis"
db_settings = {
    "host": server,
    "database": database,
    "user": username,
    "password": password,
    "port":port,
    "config_query":engine_config
}
engine = create_engine('mssql+pyodbc://{user}:{password}@{host}:{port}/{database}{config_query}'.format(**db_settings))

In [ ]:
# DBにぶち込む
table_name = "main_table"
data.to_sql(table_name, con=engine, if_exists="fail", index=False)

In [15]:
# DBにぶち込む
table_name = "test_table"
data.head().to_sql(table_name, con=engine, if_exists="replace", index=False)

In [16]:
# DBの読み込み
pd.read_sql_query("select * from test_table;", con = engine)

,traffic,address,rent,area,deposit,key_money,brokerage_fee,date,layout,layout_detail,...,general_hospital_Null_to_Zero,hospital_Null_to_Zero,park_Null_to_Zero,university_Null_to_Zero,rental_video_Null_to_Zero,cleaning_shop_Null_to_Zero,department_store_Null_to_Zero,the_number_of_rooms_Null_to_Zero,altitude,latitude
0,都営三田線春日(東京都)駅?徒歩2分丸ノ内線(池袋0荻窪)後楽園駅?徒歩4分都営大江戸線本郷...,東京都文京区本郷４丁目,7.7,19.87,77000,77000,"41,580",1985年3月,1K,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,None,None
1,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分,東京都文京区本駒込４丁目,7.9,25.01,79000,0,None,1999年3月,1K,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,None,None
2,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分千代田線 千駄木駅,東京都文京区本駒込４丁目,7.6,25.17,76000,0,None,1999年3月,1K,None,...,0.0,539.0,0.0,0.0,0.0,0.0,0.0,1,None,None
3,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分,東京都文京区本駒込４丁目,9.0,21.68,90000,90000,None,1999年3月,1K,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,None,None
4,有楽町線護国寺駅?徒歩6分有楽町線江戸川橋駅?徒歩5分丸ノ内線(池袋0荻窪)茗荷谷駅?徒歩13分,東京都文京区音羽１丁目,8.7,25.01,87000,87000,None,2013年9月,1K,洋室 7.3畳,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,None,None
